# OLYMPICS PROJECT
## About
This project focuses on analyzing the 120 years of Olympics history dataset using SQL queries. The dataset consists of two tables: "dataset_olympics.csv" and "noc_region.csv". The "dataset_olympics" table contains information about athletes, their teams, the Olympic Games they participated in, and the medals they won. The "noc_region" table provides the mapping between the National Olympic Committee codes (NOC) and the corresponding regions. By cleaning the dataset using Python and Pandas, the project aims to ensure data quality.  To extract insights based on the questions given, SQL queries will be written in module 2.

## Data Preprocessing using Python:
The initial stage of the project involves preprocessing two distinct datasets that will be used('dataset_olympics' and 'noc_region'). The data cleaning process is crucial before we can proceed to upload these datasets into the database. By doing so, we will be able to perform SQL tasks and conduct further analysis as part of the project.

## Data Cleaning Using Python:
* The data_cleaning() function reads data from a 'dataset_olympics'.csv file and performs cleaning operations on the dataset.
* It creates a copy of the data and drops specific columns, such as 'Height' and 'Weight'.
* Duplicate rows are also removed.
* Additionally, missing values in the 'Medal' column are replaced with 'Medal-less', and missing values in the 'Age' column are filled with the median age('24') from the dataset.
* Finally, the cleaned dataset is saved to a new CSV file named 'olympics_history_cleaned.csv'.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import warnings
import os
warnings.filterwarnings("ignore")


#Task1
#Loading the data
def read_data_from_csv(file_name):
    df = pd.read_csv(file_name)
    return df


# Cleaning the Dataset
def data_cleaning():
    # Read data from a CSV file
    data1_raw = read_data_from_csv('/content/drive/MyDrive/project_datasets/olympics/dataset_olympics.csv')
    data2_raw = read_data_from_csv('/content/drive/MyDrive/project_datasets/olympics/noc_region.csv')

    # Create a copy of the data
    df1 = data1_raw.copy()
    df2 = data2_raw.copy()

    # Dataset Olympics Cleaning
    # Dropping columns 'Height' and 'Weight'
    df1.drop(columns=['Height', 'Weight'], axis=1, inplace=True)

    # Drop duplicate rows
    df1.drop_duplicates(keep='first', inplace=True, ignore_index=True)

    # Replace missing values in the 'Medal' column with 'Medal-less'
    df1['Medal'].fillna('Medal-less', inplace=True)

    # Replace missing values in the 'Age' column with the median age('24') from the dataset
    df1['Age'].fillna(df1['Age'].median(), inplace=True)

    # Save the cleaned dataset to a new CSV file named 'olympics_history_cleaned.csv'
    df1.to_csv('/content/drive/MyDrive/project_datasets/olympics/olympics_history_cleaned.csv', index=False)

    # Dataset NOC Cleaning
    # Define columns to remove
    cols_remove = ['notes']

    # Drop the specified columns from the dataframe
    df2.drop(columns=cols_remove, axis=1, inplace=True)

    # Define new column names for renaming('noc_region': 'NOC' and 'reg': 'region')
    new_cols = {'noc_region': 'NOC', 'reg': 'region'}

    # Rename columns using the new column names
    df2.rename(columns=new_cols, inplace=True)

    # Save the cleaned dataset to a new CSV file named 'olympics_history_noc_regions_cleaned.csv'
    df2.to_csv('/content/drive/MyDrive/project_datasets/olympics/olympics_history_noc_regions_cleaned.csv', index=False)

def main():
  data_cleaning()

main()

## Data Analysis using MySQL
Once the data is preprocessed and cleaned it is analyzed & mined for informational insights using MySQL.

# SQL

```sql
-- Task 1
-- How many olympics games have been held?
SELECT COUNT(DISTINCT(Games)) FROM `olympics_history`

-- Task 2
-- List down all Olympics games held so far. Order the result by year.
SELECT Year, Season, City
FROM `olympics_history`
GROUP BY Games, Year, Season, City
ORDER BY Year

-- Task 3
-- Mention the total number of nations by region who participated in each olympics game?. Order the results by games.
SELECT o.Games, COUNT(DISTINCT(r.region))
FROM `olympics_history` as o
LEFT JOIN
`olympics_history_noc_regions` as r
ON o.NOC = r.NOC
GROUP BY o.Games
ORDER BY o.Games

-- Task 4
-- Which nation has participated in all of the olympic games? and order the output by first column which is fetched
SELECT n.region, COUNT(DISTINCT(o.Games)) as total_participated_games
FROM `olympics_history` as o
LEFT JOIN `olympics_history_noc_regions` as n
ON o.NOC = n.NOC
GROUP BY n.region
HAVING total_participated_games =
(SELECT COUNT(DISTINCT(Games)) FROM `olympics_history`)
ORDER BY 1


-- Task 5
-- How many unique athletes have won a gold medal in the Olympics?
SELECT COUNT(DISTINCT(Name))
FROM `olympics_history`
WHERE Medal = 'Gold'


-- Task 6
-- Which Sports were just played only once in each olympic game? and Order the output by Sports. output should include number of games.
SELECT Sport, COUNT(DISTINCT(Games)) as no_of_games, Games
FROM `olympics_history`
GROUP BY Sport
HAVING  no_of_games = 1


-- Task 7
-- Fetch the total number of sports played in each olympic games. Order by no of sports by descending.
SELECT Games, COUNT(DISTINCT(Sport)) as Total_Sports_Played
FROM `olympics_history`
GROUP BY Games
ORDER BY Total_Sports_Played DESC

-- Task 8
-- Fetch oldest athlete to win a gold medal
SELECT * FROM (SELECT Name,Sex,age, Team, Games, City, Sport, Event, Medal,
DENSE_RANK() OVER (ORDER BY Age DESC) rnk
FROM `olympics_history`
WHERE Medal = 'Gold') temp
WHERE rnk = 1

-- Task 9
-- Top 5 athletes who have won the most gold medals. Order the results by gold medals in descending.
SELECT name, team, total_gold_medals FROM (SELECT Name as name, Team as team, COUNT(Medal) as total_gold_medals, ROW_NUMBER() OVER (ORDER BY COUNT(Medal) DESC) as rnk
FROM `olympics_history`
WHERE Medal = 'Gold'
GROUP BY Name, Team) tmp
WHERE rnk <= 5

-- Task 10
-- Top 5 athletes who have won the most medals (gold/silver/bronze). Order the results by medals in descending.
SELECT Name, Team, total_medals FROM (SELECT Name, Team, Count(Medal) as total_medals, ROW_NUMBER() OVER (ORDER BY Count(Medal) DESC) as rnk
FROM `olympics_history`
WHERE Medal IN ('Gold', 'Bronze', 'Silver')
GROUP BY Name, Team) tmp
WHERE rnk <= 5

-- Task 11
-- Top 5 most successful countries in olympics. Success is defined by no of medals won.
SELECT country, total_medals, rnk FROM (SELECT r.region as country, COUNT(o.Medal) as total_medals, ROW_NUMBER() OVER (ORDER BY COUNT(o.Medal) DESC) as rnk
FROM
`olympics_history` as o
LEFT JOIN
`olympics_history_noc_regions`as r
ON o.NOC = r.NOC
WHERE o.Medal <> 'Medal-less'
GROUP BY r.region) tmp
WHERE rnk <= 5

-- Task 12
-- write your query here
-- In which Sport/event, India has won highest medals.
SELECT sport, total_medals_won FROM ( SELECT r.region, o.Sport as sport, COUNT(o.Medal) as total_medals_won, ROW_NUMBER() OVER (ORDER BY COUNT(o.Medal) DESC) as rnk
FROM
`olympics_history` as o
LEFT JOIN
`olympics_history_noc_regions` as r
ON o.NOC = r.NOC
WHERE r.region = "India" AND Medal <> 'Medal-less'
GROUP BY r.region, o.Sport) tmp
WHERE rnk = 1

-- Task 13
-- Break down all olympic games where india won medal for Hockey and how many medals in each olympic games and order the result by no of medals in descending.
SELECT o.Games, COUNT(o.Medal) as total_medals
FROM `olympics_history` as o
LEFT JOIN
`olympics_history_noc_regions` as r
ON o.NOC = r.NOC
WHERE r.region = 'India' AND Sport = 'Hockey'
GROUP BY Games
ORDER BY total_medals DESC
```


